In [ ]:
import pdfplumber
import os
import re
from operator import itemgetter

In [ ]:
rel_file_path = ''

In [ ]:
abs_file_path = os.path.abspath(rel_file_path)

In [ ]:
total_row_regex = r'Всего[\s\xa0]+к[\s\xa0]+оплате.*'
invoice_id_regex = r'(?<=Счет[\s\xa0-]фактура[\s\xa0]No)(.*)(?=от)'
invoice_date_regex = r'(?<=от)(.*)(?=\(1\))'
total_row_costs_regex = r'(([0-9]+[\s\xa0]?)+[,]?[0-9]+)'

with pdfplumber.open(abs_file_path) as pdf:
    first_page = pdf.pages[0]
    # search invoice id
    id_search_result = first_page.search(invoice_id_regex, y_tolerance=1, case=False)
    print(f'invoiceId: {id_search_result[0]['text']}')
    [page_left, _, page_right, _] = first_page.bbox
    # crop line from page and search date
    id_top, id_bottom = itemgetter('top', 'bottom')(id_search_result[0])
    first_page_cropped = first_page.crop((page_left, id_top, page_right, id_bottom))
    date_search_result = first_page_cropped.search(invoice_date_regex, y_tolerance=1, case=False)
    print(f'invoiceDate: {date_search_result[0]['text']}')
    # search invoice total row
    total_row_search_result = first_page.search(total_row_regex, y_tolerance=2, case=False)
    total_cost = re.findall(total_row_costs_regex, total_row_search_result[0]['text'])[-1][0]
    total_cost_search_result = first_page.search(total_cost, y_tolerance=2, case=False)
    print(f'totalCost: {total_cost}')
    # draw rects of results
    im = first_page.to_image(resolution=150)
    im.draw_rects([*id_search_result, *date_search_result, *total_cost_search_result])
    im.show()